> Aline's comments will be in quote blocks like this one. :)

# A first example of a numerical simulation

In this exercise, you will conduct a first instance of numerical simulation of the time evolution of a system. We will solve an advection equation of the form

$$\frac{\partial u}{\partial t} + a \frac{\partial u}{\partial x} = 0  \tag{1}$$

with $a$ being a __constant__. In subsequent exercises, more general cases will be considered, with $a$ no longer constant.

## 1 – The $a = const$ case: the mathematical problem

We would like to solve equation (1) numerically for $x  [x_0, x_f]$ with $x_0 = −2.6$, $x_f = 2.6$, periodic boundary conditions and with the initial condition:

$$u(x,t=t_0) = \cos^2 \left(\frac{6 \pi x}{5} \right) / \cosh(5x^2) \tag{2}$$

We start by representing that function in the given interval.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

import matplotlib as mpl
mpl.rc('lines', linewidth=2)

from nm_lib import nm_lib as nm

In [ ]:
def u_0(x: np.ndarray, t: float = 0) -> np.ndarray:
    r"""
    Initial condition for the advection equation.

    Parameters
    ----------
    x : `array`
        the x-axis.
    t : `float`
        the time.
    
    Returns
    -------
    `array`
        the initial condition.
    """
    return np.cos(6*np.pi*x / 5)**2 / np.cosh(5*x**2)

nx = 128 # Best number of grid points

x0 = -2.6
xf = 2.6
xx = np.linspace(x0, xf, nx) 

plt.plot(xx, u_0(xx))

> Here we see the initial function $u(x,t=t_0) = \cos^2 \left(\frac{6 \pi x}{5} \right) / \cosh(5x^2)$ in all its glory. 

## 2 – Spatial derivative.

We discretize the initial condition by subdividing the spatial domain into $nint=64$ equal intervals. The function will therefore be sampled with $nump=65$ points, namely $(u_0, u_1, u_2, ... , u_{nump−1})$ at equidistant values of the abscissa, $(x_0, x_1, x_2, ..., x_{nump−1})$. Let us calculate the spatial derivative of the function through non-centered finite differences of the form:

$$\left(\frac{\partial u}{\partial x}\right)_{x=x_i} \rightarrow \frac{u_{i+1}-u_{i}}{\Delta x}  \tag{3}$$

On the basis of the experience gained with the previous batch of exercises (ex. 1), which order of approximation can we expect now for the calculation of the derivative since we are using non-centered finite differences?

> This scheme is similar to our previously used downwind scheme, which has order of approximation 2. So this one has the same order of approximation.

## 3 – Time advance.

We want to calculate an approximation to the value of the function at times later than $t = t_0$, for instance, at time $t_0 + \Delta t$. To that end, we carry out a discretization of the time axis, calculating approximate values for the time derivative as follows:

$$\left(\frac{\partial u}{\partial t}\right)_{\underset{t=t_0}{x=x_i}} \rightarrow \frac{u_i(t_0 + \Delta t)-u_i(t_0)}{\Delta t}\tag{4}$$

Using now the differential equation (1) and expression (3), we can calculate an approximate value for the function $u$ at $x_i$ and time $t_0 + \Delta t$ as follows:

$$u_i(t_0+\Delta t) = u_i(t_0) - a \frac{u_{i+1}(t_0) - u_i(t_0)}{\Delta x}\Delta t  \tag{5}$$

Using (5), calculate $u_i$ at time $t_0 + \Delta t$ at all points of the sample excluding the rightmost point (i.e., excluding $x_{nump−1}$). Use  $\Delta t = 0.98 \Delta x/|a|$ and a = −1. Fill in `nm_lib` the functions `step_adv_burgers` and `cfl_adv_burger`. 

In [ ]:
a = -1
nx = 128 

xx = np.linspace(x0, xf, nx) 
dt, rhs = nm.step_adv_burgers(xx, u_0(xx), a=a) 

plt.plot(xx, u_0(xx), label='t=0')

u1 = u_0(xx) + dt * rhs 
plt.plot(xx, u1, label='t=1')
plt.legend()

> Here we have used the `step_adv_burgers` method and manually calculated the next timestep in order to verify the method. We see that the solution moves to the left after one timestep.

## 4 – The boundaries.

To calculate the function at the rightmost point of the interval, we use a periodicity condition on $u$: $u_{nump−1}(t_0 + \Delta t) = u_0(t_0 + \Delta t)$. Consider cutting the ill-calculated (or missing) grid points and using `NumPy.pad` to add different boundary conditions. Use `wrap`

In [ ]:
nt = 200
nx = 128 

xx = np.linspace(x0, xf, nx) 
t, unnt = nm.evolv_adv_burgers(xx, u_0(xx), nt, a=-1)

plt.plot(xx, u_0(xx), label='t = 0.00')
plt.plot(xx, unnt[:, 150], label=f't = {t[150]:.2f}')
plt.legend()

> Having implemented the periodic boundary conditions, we can already see how the numerical solution *diffuses* in time. 

## 5 – Subsequent steps in time.

Having calculated the values $u_i$, $i = 0, 1, 2, ...,nump−1,$ at time $t_0+\Delta t$, we can carry out another step in time of size $\Delta t$, following exactly the method just explained. In general, if we have calculated the value of the $u$ function at $x_i$ and time $n\Delta t$, which we will call $u^n_i$, we can carry out the next step in time, of size $\Delta t$, through the expression:

$$u_i^{n+1} = u_i^n - a \frac{u_{i+1}^n - u_i^n}{\Delta x}\Delta t  \tag{6}$$

Periodic boundary conditions must be applied, as explained in 4 above, to finish the calculation at each timestep. Fill in nm_lib the function `evolv_adv_burgers`. 

Carry out many steps in time so you can clearly understand the mathematical nature of the solution of the equation. For example, in Python, you can _matplotlib.animation_ to see the evolution. 

In [ ]:
nt = 300
nx = 100
xx = np.linspace(x0, xf, nx) 
t, unnt = nm.evolv_adv_burgers(xx, u_0(xx), nt, a=a)

from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 3))

def init(): 
    axes.plot(xx,unnt[:,0])

def animate(i):
    axes.clear()
    axes.plot(xx,unnt[:,i])
    axes.set_title('t=%.2f'%t[i])
    axes.set_ylim(-0.05, 1.05)
    
anim = FuncAnimation(fig, animate, interval=10, frames=nt, init_func=init)
plt.close()
HTML(anim.to_jshtml())

> For a better result, I would increase `nx`, but this will make rendering the animation a lot slower.


> We see that the function gets slightly deformed after many timesteps. Increasing the amount of grid points minimizes this error, but it is still easy to see that the solution deforms after a while.

## 6 – Comparison with the exact solution.

Through one of the theory sessions, we know the exact solution of eq. (1) for the initial condition (2). Draw the exact solution on top of the numerical one using a dashed line. Then, explain the mathematical behavior of the solution. __Important note__: consider that the initial condition is eq (2) in $[x_0, x_f]$, _with periodic conditions at the boundaries_. In other words, the initial condition consists of an infinite repetition of the function (2) you represented between $[x_0, x_f]$ next to each other. Use this fact when comparing your numerical solution with the analytical one. __Hint__: if you consider points starting within the interval $(x_0, x_f)$ and moving with speed $a$, they will go outside the domain after some time. Use mod operator (\% in Python) to bring them back into the domain respecting the periodicity of the problem or _numpy.pad_ to pad ghost points, i.e., points out of the numerical domain, which will allow defining the boundaries, at both ends of the numerical domain. NumPy.pad allows various types of boundaries.

In [ ]:
def analytical(
    xx: np.ndarray, t: np.ndarray) -> np.ndarray:
    r"""
    Analytical solution to the advection equation.

    Requires
    ----------
    u(xx) : `function`

    Parameters
    ----------
    xx : `array`
         the x-axis.
    t  : `array`
         the time. 

    Returns
    ------- 
    `array`
        the analytical solution over the times `t`.
    """

    u_an = np.zeros((len(xx), len(t)))
    x_new = np.zeros((len(xx), len(t)))

    for i in range(len(t)):

        x_new[:, i] = ((xx - a * t[i] ) - x0) % (xf - x0) + x0
        u_an[:, i] = u_0(x_new[:, i])
    
    return u_an

In [ ]:
nt = 300
nx = 100
xx = np.linspace(x0, xf, nx) 

t, unnt = nm.evolv_adv_burgers(xx, u_0(xx), nt, a=a)
u_an = analytical(xx, t)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 3))

def init(): 
    axes.plot(xx, unnt[:, 0])
    axes.plot(xx, u_an[:, 0], ls='--')

def animate(i):
    axes.clear()
    axes.plot(xx, u_an[:, i], c='k', label='analytical')
    axes.plot(xx, unnt[:, i], ls='--', label='numerical')
    axes.set_title('t=%.2f'%t[i])
    axes.set_ylim(-0.05, 1.05)
    axes.legend(loc='upper right')
    
anim = FuncAnimation(fig, animate, interval=10, frames=nt, init_func=init)
plt.close()
HTML(anim.to_jshtml())

> We see that the numerical solution diffuses compared to the numerical solution, which has the same shape as the function moves.

Add a CI/CD pipeline to run this test and validate each push commit. This lets us know if a specific submitted change damages the existing code. For this, fill in ./github/workflows/test.yml

> The test function is found on GitHub on the path `alinebrunvoll/nm_lib/nm_lib/test/test_ex_2b.py`. It runs automatically for each new commit to the `nm_lib` repository. I also tried at one point to use a test that wouldn't pass, just to see what would happen. 

## 7 – Resolution increase in space and time. 

Let us repeat the calculation of the previous paragraphs, increasing the number of space intervals by factors of 2. Check if the numerical solution gets increasingly close to the analytical solution. Important: the comparison must be made _for the same values of $x$ and $t$ for all resolutions you check_. Choose a fixed time when the traveling function has already gone through the whole domain a few times. 

In [ ]:
nt = 1000

nx_1 = 100
nx_2 = 400

xx_1 = np.linspace(x0, xf, nx_1)
xx_2 = np.linspace(x0, xf, nx_2)

t_1, unnt_1 = nm.evolv_adv_burgers(xx_1, u_0(xx_1), nt, a=a)
t_2, unnt_2 = nm.evolv_adv_burgers(xx_2, u_0(xx_2), nt, a=a)

u_an_1 = analytical(xx_1, t_1)
u_an_2 = analytical(xx_2, t_2)


# finding the (roughly) same time for both solutions
t_ind_1 = np.where(np.isclose(t_1, 20.0, 0.4))[0][0]
t_ind_2 = np.where(np.isclose(t_2, 20.0, 0.4))[0][0]


# plotting
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

ax[0].plot(xx_1, u_an_1[:, t_ind_1], label='analytical')
ax[0].plot(xx_1, unnt_1[:, t_ind_1], ls='--', label='numerical')
ax[0].set_title(f'nx={nx_1}, t={t_1[t_ind_1]:.2f}')
ax[0].legend()

ax[1].plot(xx_2, u_an_2[:, t_ind_2], label='analytical')
ax[1].plot(xx_2, unnt_2[:, t_ind_2], ls='--', label='numerical')
ax[1].set_title(f'nx={nx_2}, t={t_2[t_ind_2]:.2f}')
ax[1].legend()

> We see clearly that when we increase the amount of gridpoints `nx`, the numerical solution diffuses more slowly, and thus stays closer to the analytical solution. 